In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [2]:
browser=webdriver.Chrome()

In [3]:
browser.get('https://www.tripadvisor.in/Hotels-g186338-London_England-Hotels.html')


In [4]:
accept = browser.find_element(By.ID, "onetrust-accept-btn-handler")
accept.click()

In [6]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [45]:
list_of_numbers = str(list(range(1, 101)))
hotels = []
for name in soup.findAll('a',{'data-clicksource':'HotelName'}):
      hotels.append(name.text.strip(list_of_numbers + '.'))
hotels[:5]


['YOTEL London Shoreditch',
 'The Montcalm Royal London House',
 'Park Grand London Hyde Park',
 'The Resident Covent Garden',
 'Park Grand Paddington Court']

In [26]:
ratings = []
for rating in soup.findAll('a',{'class':'ui_bubble_rating'}):
      if rating['alt'] == "5 of 5 bubbles":
            rating = 10
      elif rating['alt'] == "4.5 of 5 bubbles":
            rating = 9
      elif rating['alt'] == "4 of 5 bubbles":
            rating = 8
      elif rating['alt'] == "3.5 of 5 bubbles":
            rating = 7
      elif rating['alt'] == "3 of 5 bubbles":
            rating = 6
      else:
            rating = 5
      ratings.append(rating)
len(ratings)

34

In [67]:
reviews = []
for review in soup.findAll('a',{'class':'review_count'}):
  reviews.append(review.text.replace('reviews','').replace(',','').strip())
len(reviews)

34

In [68]:
price = []
for p in soup.findAll('div',{'data-sizegroup':'mini-meta-price'}):
  price.append(p.text.replace('€','').replace(',','').strip()) 
price[:5]
len(price)

34

In [69]:
d1 = {'Hotel':hotels,'Ratings':ratings,'No_of_Reviews':reviews,'Price, €':price}
df = pd.DataFrame.from_dict(d1)

In [70]:
df

,Hotel,Ratings,No_of_Reviews,"Price, €"
0,YOTEL London Shoreditch,9,685,112
1,The Montcalm Royal London House,10,3356,207
2,Park Grand London Hyde Park,8,3362,159
3,The Resident Covent Garden,10,1243,302
4,Park Grand Paddington Court,9,3212,145
5,Travelodge London City hotel,9,2378,59
6,"The Biltmore Mayfair, LXR Hotels & Resorts",8,173,930
7,Canopy by Hilton London City,10,424,194
8,The Montcalm London Marble Arch,10,3812,473
9,Hard Rock Hotel London,9,2243,386


In [71]:
df.to_csv("tripadvisor_london_list.csv")

In [73]:
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

In [74]:
tripadvisor = pd.read_csv('tripadvisor_london_list.csv')
tripadvisor.head()


,Unnamed: 0,Hotel,Ratings,No_of_Reviews,"Price, €"
0,0,YOTEL London Shoreditch,9,685,112
1,1,The Montcalm Royal London House,10,3356,207
2,2,Park Grand London Hyde Park,8,3362,159
3,3,The Resident Covent Garden,10,1243,302
4,4,Park Grand Paddington Court,9,3212,145


In [75]:
tripadvisor.to_sql('tripadvisor', conn, if_exists='append', index = False)


34

In [76]:
booking = pd.read_csv('bookingcom_london_list.csv')
booking.head()

,Unnamed: 0,Hotel,Ratings,No_of_Reviews,"Price, €"
0,0,One Hundred Shoreditch,8.6,1622,2422
1,1,Stunning New Development in Barking,7.1,25,771
2,2,"St. James' Court, A Taj Hotel, London",8.7,3524,2596
3,3,Bayswater by CAPITAL,6.9,108,1454
4,4,Nobu Hotel London Portman Square,8.5,865,5336


In [77]:
booking.to_sql('booking_com', conn, if_exists='append', index = False)

25